In [1]:
import pandas as pd

## Import Feature Engineered Data

In [2]:
train_df = pd.read_csv('data/inter/train_clean.csv')
test_df = pd.read_csv('data/inter/test_clean.csv')

## Feature Selection

In [3]:
drop_cols = ['Dates', 'DayOfWeek', 'Address', 'X', 'Y', 'Month' ]
train_df = train_df.drop(drop_cols+['Descript', 'Resolution'], axis=1)
test_df = test_df.drop(drop_cols, axis=1)

In [4]:
train_df.columns

Index(['PdDistrict', 'Year', 'IsDay', 'MonthSin', 'MonthCos', 'DayOfWeekSin',
       'DayOfWeekCos', 'Category_ARSON', 'Category_ASSAULT',
       'Category_BAD CHECKS', 'Category_BRIBERY', 'Category_BURGLARY',
       'Category_DISORDERLY CONDUCT', 'Category_DRIVING UNDER THE INFLUENCE',
       'Category_DRUG/NARCOTIC', 'Category_DRUNKENNESS',
       'Category_EMBEZZLEMENT', 'Category_EXTORTION',
       'Category_FAMILY OFFENSES', 'Category_FORGERY/COUNTERFEITING',
       'Category_FRAUD', 'Category_GAMBLING', 'Category_KIDNAPPING',
       'Category_LARCENY/THEFT', 'Category_LIQUOR LAWS', 'Category_LOITERING',
       'Category_MISSING PERSON', 'Category_NON-CRIMINAL',
       'Category_OTHER OFFENSES', 'Category_PORNOGRAPHY/OBSCENE MAT',
       'Category_PROSTITUTION', 'Category_RECOVERED VEHICLE',
       'Category_ROBBERY', 'Category_RUNAWAY', 'Category_SECONDARY CODES',
       'Category_SEX OFFENSES FORCIBLE', 'Category_SEX OFFENSES NON FORCIBLE',
       'Category_STOLEN PROPERTY', 'Ca

In [5]:
test_df.columns

Index(['Id', 'PdDistrict', 'Year', 'IsDay', 'MonthSin', 'MonthCos',
       'DayOfWeekSin', 'DayOfWeekCos'],
      dtype='object')

## Separate Features (X) and Targets (Y):

In [10]:
feature_cols = ['Year', 'MonthSin', 'MonthCos', 'DayOfWeekSin', 'DayOfWeekCos', 'IsDay', 'PdDistrict']
train_x = train_df[feature_cols]
train_y = train_df.drop(feature_cols, axis=1)
test_ids = test_df.Id # Save Ids for submission
test_x  = test_df[feature_cols]

In [11]:
train_x.sample(3)

,Year,MonthSin,MonthCos,DayOfWeekSin,DayOfWeekCos,IsDay,PdDistrict
405611,2009,-1.000000,-1.836970e-16,0.433884,-0.900969,1,7
601888,2006,-0.866025,5.000000e-01,0.974928,-0.222521,1,7
685058,2005,-0.866025,-5.000000e-01,-0.974928,-0.222521,0,7


In [12]:
test_x.sample(3)

,Year,MonthSin,MonthCos,DayOfWeekSin,DayOfWeekCos,IsDay,PdDistrict
362481,2010,0.866025,-0.500000,-0.781831,0.623490,1,7
191766,2012,-0.500000,0.866025,0.974928,-0.222521,1,7
535496,2007,-0.500000,0.866025,0.974928,-0.222521,1,3


In [13]:
train_y.columns

Index(['Category_ARSON', 'Category_ASSAULT', 'Category_BAD CHECKS',
       'Category_BRIBERY', 'Category_BURGLARY', 'Category_DISORDERLY CONDUCT',
       'Category_DRIVING UNDER THE INFLUENCE', 'Category_DRUG/NARCOTIC',
       'Category_DRUNKENNESS', 'Category_EMBEZZLEMENT', 'Category_EXTORTION',
       'Category_FAMILY OFFENSES', 'Category_FORGERY/COUNTERFEITING',
       'Category_FRAUD', 'Category_GAMBLING', 'Category_KIDNAPPING',
       'Category_LARCENY/THEFT', 'Category_LIQUOR LAWS', 'Category_LOITERING',
       'Category_MISSING PERSON', 'Category_NON-CRIMINAL',
       'Category_OTHER OFFENSES', 'Category_PORNOGRAPHY/OBSCENE MAT',
       'Category_PROSTITUTION', 'Category_RECOVERED VEHICLE',
       'Category_ROBBERY', 'Category_RUNAWAY', 'Category_SECONDARY CODES',
       'Category_SEX OFFENSES FORCIBLE', 'Category_SEX OFFENSES NON FORCIBLE',
       'Category_STOLEN PROPERTY', 'Category_SUICIDE',
       'Category_SUSPICIOUS OCC', 'Category_TREA', 'Category_TRESPASS',
       'Cate

## Prediction